In [1]:
from dsipts import Categorical,TimeSeries, RNN


In [2]:
settimana = Categorical('settimanale',1,[1,1,1,1,1,1,1],7,'multiplicative',[0.9,0.8,0.7,0.6,0.5,0.99,0.99])

##montly, additive (here there are only 5 month)
mese = Categorical('mensile',1,[31,28,20,10,33],5,'additive',[10,20,-10,20,0])

##spot categorical variables: in this case it occurs every 100 days and it lasts 7 days adding 10 to the original timeseries
spot = Categorical('spot',100,[7],1,'additive',[10])

##initizate a timeseries object
ts = TimeSeries('prova')

In [3]:
ts.generate_signal(noise_mean=1,categorical_variables=[settimana,mese,spot],length=5000,type=0)


In [25]:
dataset = ts.dataset
dataset['x_num'] = 0.0
dataset.loc[:dataset.shape[0]-10,'x_num']= dataset['x_num'].values[9:]

In [27]:
cat_var_past = ts.cat_var
cat_var_fut = ts.cat_fut
num_var_past = ts.num_var+['x_num']
num_var_fut = []
group = None
time_var = 'time'
target = ['signal']


['signal']

In [63]:
import pandas as pd
from typing import Union
def extend_time_df(x:pd.DataFrame,freq:Union[str,int],group:Union[str,None]=None,global_minmax:bool=False)-> pd.DataFrame:
    """Utility for generating a full dataset and then merge the real data

    Args:
        x (pd.DataFrame): dataframe containing the column time
        freq (str): frequency (in pandas notation) of the resulting dataframe
        group (string or None): if not None the min max are computed by the group column, default None
        global_minmax (bool): if True the min_max is computed globally for each group. Usually used for stacked model
    Returns:
        pd.DataFrame: a dataframe with the column time ranging from thr minumum of x to the maximum with frequency `freq`
    """

    if group is None:

        if isinstance(freq,int):
            empty = pd.DataFrame({'time':list(range(x.time.min(),x.time.max(),freq))})
        else:
            empty = pd.DataFrame({'time':pd.date_range(x.time.min(),x.time.max(),freq=freq)})

    else:
        
        if global_minmax:
            _min = pd.DataFrame({group:x[group].unique(),'time':x.time.min()})
            _max = pd.DataFrame({group:x[group].unique(),'time':x.time.max()})

        else:
            _min = x.groupby(group).time.min().reset_index()
            _max = x.groupby(group).time.max().reset_index()
        empty = []
        for c in x[group].unique():
            if isinstance(freq,int):
                empty.append(pd.DataFrame({group:c,'time':np.arange(_min.time[_min[group]==c].values[0],_max.time[_max[group]==c].values[0],freq)}))

            else:
                empty.append(pd.DataFrame({group:c,'time':pd.date_range(_min.time[_min[group]==c].values[0],_max.time[_max[group]==c].values[0],freq=freq)}))
            
        empty = pd.concat(empty,ignore_index=True)
    return empty


In [68]:
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import lightning as L

class PandasTSDataSet:
    def __init__(self, df,time_var, cat_var_past, cat_var_fut, num_var_past, num_var_fut,target, group = None, metadata={},check_holes_and_duplicates=True,check_past=True):
        
        numerica_var = list(set(num_var_past).union(set(num_var_fut)).union(set(target)))
        df_norm = df[numerica_var+[time_var]].copy()
        label_encoders = {}
        for c in set(cat_var_past).union(set(cat_var_fut)):
            label_encoders[c] = LabelEncoder()
            df_norm[c] = label_encoders[c].fit_transform(df[c])
        if group is not None:
            label_encoders[group] = LabelEncoder()
            df_norm[group] = label_encoders[group].fit_transform(df[group])
        
        if check_holes_and_duplicates:
            df_norm.drop_duplicates(subset=['time'] if group is None else [group,'time'],  keep='first', inplace=True, ignore_index=True)
            print(df_norm.shape)
            if group is None:
                differences = df_norm.time.diff()[1:]
            else:
                differences = df_norm[df_norm[group]==df_norm[group].unique()[0]].time.diff()[1:]
    
            if isinstance(df_norm.time[0], datetime):
                freq = pd.to_timedelta(differences.min())   
            else:
                if int(df_norm.time[0])==df_norm.time[0]: ##ONLY THINK THAT WORKS IN GENERAL
                    freq = int(differences.min())
                else:
                    raise TypeError("time must be integer or datetime")
            metadata['fraq'] = freq 
            if differences.nunique()>1:
                df_norm = extend_time_df(df_norm,freq,group).merge(dataset,how='left')
                print(df_norm.shape)
        
        else:
            self.freq =  dataset.time.diff()[1:].min() ##care there can be holes in data!

        if check_past:
            num_var_past = list(set(num_var_past).union(set(target)))
        if group is None:
            df_norm = df_norm.sort_values(by=time_var).reset_index()
        else:
            df_norm = df_norm.sort_values(by=[time_var,group]).reset_index()

        self.df = df_norm
        self.metadata = metadata   
        self.target = target
        self.num_var_past = num_var_past
        self.num_var_fut = num_var_fut
        self.cat_var_past = cat_var_past
        self.cat_var_fut = cat_var_fut
        self.length = 1 if group is None else df[group].nunique()
        self.group = group
    def __getitem__(self, idx):
        # Returns only tensors, no metadata
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        if self.group is None:
            df = self.df
        else:
            df = self.df[self.df.group==idx] ## ANOTHER COMMENT: IDX?? what if we have 10 timeseries with groups??
            
        result = {
            #'t': tensor(...), ##FIRST COMMENT: TIMESTAMP CANT be cast as tensor! 
            'y': torch.tensor(df[self.target].values,dtype=torch.float32,device=device), ##SECOND COMMENT device
            'x_past': torch.tensor(df[self.num_var_past].values,dtype=torch.float32,device=device), ##SECOND COMMENT device
            #'t_f': tensor(...), ?? WHAT IS THAT?
        
            }
        if self.group is not None:
             result['group']= torch.tensor(idx,device=device,dtype=torch.long) 
        if len(self.num_var_fut)>0:
             result['num_var_fut']= torch.tensor(df[self.num_var_fut].values,dtype=torch.float32,device=device)
        if len(self.cat_var_fut)>0:
             result['cat_var_fut']= torch.tensor(df[self.cat_var_fut].values,dtype=torch.long,device=device)
        if len(self.cat_var_fut)>0:
             result['cat_var_fut']= torch.tensor(df[self.cat_var_fut].values,dtype=torch.long,device=device)
        return result
    def get_metadata(self):
        return self.metadata

# Layer D2


class DecoderEncoderDataModule(L.LightningDataModule):
    def __init__(self, d1_dataset, batch_size=32, num_workers=4,metadata=None):
        super().__init__()
        # initialize other  params
        self.d1_dataset = d1_dataset
        self.batch_size = batch_size
        self.metadata = None  # Will be set in setup() NO! THhe metadata for training are not related with metadata of the data!
    
    def setup(self, stage=None):
        # Get metadata from D1 layer during setup
        #self.metadata = self.d1_dataset.get_metadata() ##NO SEE COMMENT BEFORE!
        
        if stage == 'fit' or stage is None: 
            ##CLEAR FOR ME IN CASE OF SINGLE TIMESERIES (EG PANDAS LOADER) not clear in case of chunk of data of the same 
            ## do we suppose to have NO HOLES in data? do we check it runtime?
            pass
        
        if stage == 'test' or stage is None:
            # Any test-specific setup
            pass
            
    def train_dataloader(self):
        return DataLoader(
            self.d1_dataset,
            batch_size=self.batch_size,
            **other_params
        )

# Layer M
class DecoderEncoderModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.metadata = None
        
    def setup(self, stage=None):
        # Get metadata from datamodule during setup
        self.metadata = self.trainer.datamodule.metadata
        
        # Initialize layer T model using metadata
    
    def forward(self, x):
     # forward logic
        pass

In [69]:
data_module_metadata = dict(
  perc_train= 0.7,
  perc_valid= 0.1,
  range_train= None,
  range_validation= None ,
  range_test= None,
  shift= 0,
  starting_point= None,
  skip_step= 1,
  past_steps=16,
  future_steps= 16,
  scaler= 'sklearn.preprocessing.StandardScaler()')

In [70]:
cat_var_past = ts.cat_var
cat_var_fut = ts.cat_var
num_var_past = ts.num_var+['x_num']
num_var_fut = []
group = None
time_var = 'time'
target = ['signal']


In [71]:
ds = PandasTSDataSet(dataset,'time', 
                     cat_var_past, 
                     cat_var_fut, 
                     num_var_past,
                     num_var_fut,
                     target,
                     group = None, 
                     metadata={}
                     ,check_holes_and_duplicates=True,check_past=True)
        

(5000, 6)


In [76]:
next(iter(ds))['y'].shape

torch.Size([5000, 1])